In [1]:
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

In [2]:
res=requests.get('https://quotes.toscrape.com/')
soup=BeautifulSoup(res.text,'html.parser')

In [3]:
data=[]
for sp in soup.find_all('div',class_ = 'quote'):
    quote      = sp.find('span',class_ = 'text').text[1:-1]
    
    author    = sp.find('small').text
    
    author_id = sp.find('a').get('href')
    
    tags=[]
    for tag in sp.find_all('a',class_ = 'tag'):
        tags.append(tag.get_text(strip=True))
    tags      =  ','.join(tags) 
    
    data.append([quote,author,author_id,tags])


In [4]:
data = []  # this will store ALL quotes data as a list of rows

# loop through every quote block on the page
for sp in soup.find_all('div', class_='quote'):

    # extract the quote text (span.text contains curly quotes “ ” on this site)
    # [1:-1] removes the first and last character (usually the quotes)
    quote = sp.find('span', class_='text').text[1:-1]

    # extract author name (example: Albert Einstein)
    author = sp.find('small').text

    # extract author profile link (relative URL like /author/Albert-Einstein)
    author_id = sp.find('a').get('href')

    # extract all tag names for this quote
    tags = []  # temporary list for tags of THIS quote only
    for tag in sp.find_all('a', class_='tag'):
        tags.append(tag.get_text(strip=True))  # get clean tag text

    # convert list of tags into single string: "change,inspirational,life"
    tags = ','.join(tags)

    # store this quote row in data list
    data.append([quote, author, author_id, tags])


In [5]:
data=[]
for page in tqdm(range(1,11)):
    link='https://quotes.toscrape.com/page/'+str(page)
    res=requests.get(link)
    soup=BeautifulSoup(res.text,'html.parser')

    for sp in soup.find_all('div',class_ = 'quote'):
        quote      = sp.find('span',class_ = 'text').text[1:-1]
        author    = sp.find('small').text
        author_id = sp.find('a').get('href')
        
        tags=[]
        for tag in sp.find_all('a',class_ = 'tag'):
            tags.append(tag.get_text(strip=True))
        tags      =  ','.join(tags) 
        
        data.append([quote,author,author_id,tags])   

100%|██████████████████████████████████████████████████████████████████████████████| 10/10 [00:17<00:00,  1.80s/it]


In [6]:
len(data)

100

In [7]:
df = pd.DataFrame(data,columns = ['quote','author','author_id','tags'])

In [8]:
df.head()

,quote,author,author_id,tags
0,The world as we have created it is a process o...,Albert Einstein,/author/Albert-Einstein,"change,deep-thoughts,thinking,world"
1,"It is our choices, Harry, that show what we tr...",J.K. Rowling,/author/J-K-Rowling,"abilities,choices"
2,There are only two ways to live your life. One...,Albert Einstein,/author/Albert-Einstein,"inspirational,life,live,miracle,miracles"
3,"The person, be it gentleman or lady, who has n...",Jane Austen,/author/Jane-Austen,"aliteracy,books,classic,humor"
4,"Imperfection is beauty, madness is genius and ...",Marilyn Monroe,/author/Marilyn-Monroe,"be-yourself,inspirational"


In [9]:
df['author_link']='https://quotes.toscrape.com/'+df['author_id']

In [10]:
df.head()

,quote,author,author_id,tags,author_link
0,The world as we have created it is a process o...,Albert Einstein,/author/Albert-Einstein,"change,deep-thoughts,thinking,world",https://quotes.toscrape.com//author/Albert-Ein...
1,"It is our choices, Harry, that show what we tr...",J.K. Rowling,/author/J-K-Rowling,"abilities,choices",https://quotes.toscrape.com//author/J-K-Rowling
2,There are only two ways to live your life. One...,Albert Einstein,/author/Albert-Einstein,"inspirational,life,live,miracle,miracles",https://quotes.toscrape.com//author/Albert-Ein...
3,"The person, be it gentleman or lady, who has n...",Jane Austen,/author/Jane-Austen,"aliteracy,books,classic,humor",https://quotes.toscrape.com//author/Jane-Austen
4,"Imperfection is beauty, madness is genius and ...",Marilyn Monroe,/author/Marilyn-Monroe,"be-yourself,inspirational",https://quotes.toscrape.com//author/Marilyn-Mo...


In [11]:
df.to_csv('Quotes.csv',index=False)
